In [ ]:
# install pytorch 
!pip install --pre torch torchvision torchaudio -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html -U

In [ ]:
# test torch version and CUDA device
import torch
print(torch.__version__)
a = torch.Tensor([1]).cuda()
print(a)

1.12.0.dev20220228+cu102
tensor([1.], device='cuda:0')


In [ ]:
# install pycocotools
!pip install pycocotools

In [ ]:
# mount google drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# change to project home directory
import os
os.chdir("/content/gdrive/My Drive/object_detection_frcnn_mscoco_boilerplate")

In [ ]:
pwd

'/content/gdrive/My Drive/object_detection_frcnn_mscoco_boilerplate'

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
from custom_dataset_mscoco import myOwnDataset
import utils
from engine import train_one_epoch, evaluate

In [ ]:
import pprint
pp = pprint.PrettyPrinter()

In [ ]:
# path to data and annotation
data_dir = 'data'
annotation = 'data/result.json'

In [ ]:
# input transformations

import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std= [0.229, 0.224, 0.225])

# train_transform = transforms.Compose([
#   transforms.Resize(227),
#   transforms.CenterCrop(227),
#   transforms.ToTensor(),
#   normalize
#  ])

# validation_transform = transforms.Compose([
#   transforms.ToTensor()
#  ])

# test_transform = transforms.Compose([
#   transforms.ToTensor()
#  ])

# create dataset
dataset_train = myOwnDataset(root=data_dir,
                          annotation=annotation,
                          transforms=get_transform(train=True)
                          )
dataset_validation = myOwnDataset(root=data_dir,
                          annotation=annotation,
                          transforms=get_transform(train=False)
                          )

dataset_test = myOwnDataset(root=data_dir,
                          annotation=annotation,
                          transforms=get_transform(train=False)
                          )

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
# split the dataset in train, validation and test set
indices = torch.randperm(len(dataset_train)).tolist()
dataset_train = torch.utils.data.Subset(dataset_train, indices[:-64])
print(len(dataset_train))
dataset_validation = torch.utils.data.Subset(dataset_validation, indices[-64:-32])
print(len(dataset_validation))
dataset_test = torch.utils.data.Subset(dataset_test, indices[-32:])
print(len(dataset_test))

In [ ]:
# define training, validation and test data loaders
data_loader_train = torch.utils.data.DataLoader(
    dataset_train, batch_size=5, shuffle=True, num_workers=1,
    collate_fn=utils.collate_fn)

data_loader_validation = torch.utils.data.DataLoader(
    dataset_validation, batch_size=5, shuffle=False, num_workers=1,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=5, shuffle=False, num_workers=1,
    collate_fn=utils.collate_fn)

In [ ]:
# select device (whether GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# # DataLoader is iterable over Dataset
# # check image_id, labels, box dimentions
# for imgs, annotations in data_loader:
#     print('image_id: ', annotations[0]['image_id'])
#     print('labels: ', annotations[0]['labels'])
#     print('boxes: ', annotations[0]['boxes'])
#     print('-----------------------------------------')

In [ ]:
def get_model_instance(num_classes):
    # load a model instance pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # print(model)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
# load model
num_classes = 14
# get the model using our helper function
model = get_model_instance(num_classes)

In [ ]:
# move model to the right device
model.to(device)

In [ ]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# optimizer = torch.optim.Adam(params, lr=0.005, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
# let's train it for 10 epochs
num_epochs = 20

In [ ]:
print("================= TRAIN and VALIDATION ===========================")
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_validation, device=device)
print("That's it!")

In [ ]:
torch.save(model.state_dict(), 'chkpnt1.pth')

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn()

In [ ]:
in_features = model.roi_heads.box_predictor.cls_score.in_features

In [ ]:
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
model.load_state_dict(torch.load('chkpnt1.pth'))

<All keys matched successfully>

In [ ]:
model.eval()

In [ ]:
print("================= INFERENCE ===========================")
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in data_loader_test:
        print("-------batch------------")
        pp.pprint(labels)
        outputs = model(images)
        pp.pprint(outputs)